In [4]:
 import numpy as mp
import pandas as pd

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [6]:
df=pd.read_csv('train.csv')

In [7]:
  df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [8]:
#Test/Train Split
X_train,X_test,Y_train,Y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [9]:
X_train.sample(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
349,3,male,42.0,0,0,8.6625,S
449,1,male,52.0,0,0,30.5000,S
454,3,male,NaN,0,0,8.0500,S


In [10]:
Y_train.sample(3)

775    0
683    0
693    0
Name: Survived, dtype: int64

In [11]:
#imputation transformer
trf1=ColumnTransformer([('impute_age',SimpleImputer(),[2]),
                       ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
                       ],remainder='passthrough')

In [12]:
#One Hot Encoding
trf2=ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(handle_unknown='ignore'),[1,3])
],remainder='passthrough')

In [13]:
#Scaling
trf3=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [14]:
#Feature Selection
trf4=SelectKBest(score_func=chi2,k=8)


In [15]:
#Train the model
trf5=DecisionTreeClassifier()

Create Pipeline

In [16]:
pipe=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

Pipeline vs make_pipeline

Pipeline requires naming of steps,make_pipeline does not
(Same implies for ColumnTransformer vs make_column_transformer)

In [17]:
#Alternate Syntax
pipe=make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [18]:
#Train
pipe.fit(X_train,Y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  [1, 3])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x00000221138832E0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

Explore the pipeline

In [19]:
pipe.steps[0][1].transformers_[1][1].statistics_


array(['S'], dtype=object)

In [20]:
#Predict
y_pred=pipe.predict(X_test)

In [21]:
y_pred

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1], dtype=int64)

In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,y_pred)

0.7932960893854749

Cross Validation using Pipeline

In [23]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,Y_train,cv=5,scoring='accuracy').mean()

0.786644341573919

Grid Search using Pipeline

In [24]:
params={
    'trf5_max_depth':[1,2,3,4,5,None]
}

In [26]:
from sklearn.model_selection import GridSearchCV

# Updated parameter grid with correct step names
params = {
    'decisiontreeclassifier__max_depth': [3, 5, 10],
    'decisiontreeclassifier__min_samples_split': [2, 5, 10],
    'selectkbest__k': [5, 8, 10]
}

# Initialize GridSearchCV with the updated params
grid = GridSearchCV(pipe, param_grid=params, cv=5, scoring='accuracy')

# Fit to training data
grid.fit(X_train, Y_train)

# Print the best parameters and score
print("Best Parameters:", grid.best_params_)
print("Best Cross-Validation Score:", grid.best_score_)


Best Parameters: {'decisiontreeclassifier__max_depth': 3, 'decisiontreeclassifier__min_samples_split': 2, 'selectkbest__k': 10}
Best Cross-Validation Score: 0.821609376538954


In [27]:
grid.best_score_

0.821609376538954

In [28]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))